Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.

# 1. Подготовка

Загрузим библиотеки и данные.

In [1]:
import pandas as pd
from pymystem3 import Mystem
import re 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Создадим функции, очищающие текст от символов и лемматизирующие слова.

In [3]:
def clear_text(text):
    re_text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    clear = " ".join(re_text.split()) 
    return clear

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemm_text

Применим наши функции.

In [4]:
comments['clear'] = comments['text'].apply(lambda x: clear_text(x))
comments.head()

,text,toxic,clear
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [5]:
comments['lemmed'] = comments['clear'].apply(lambda x: lemmatize(x))
comments.head()

,text,toxic,clear,lemmed
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...,You sir are my hero Any chance you remember wh...


# 2. Обучение

In [6]:
#разобьем текст на выборки
train, test = train_test_split(comments, test_size=0.25, random_state=42)

#созадим корпус слов из обучающей выборки
corpus1 = train['lemmed'].values.astype('U')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

# подсчитаем TF-IDF для обучающей выборки
counter = TfidfVectorizer(stop_words=stopwords)

tf_idf1 = counter.fit_transform(corpus1) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# подсчитаем TF-IDF для тестовой выборки
corpus2 = test['lemmed'].values.astype('U')
tf_idf2 = counter.transform(corpus2) 

Обучим модели случайного леса и линейной регрессии.

In [8]:
model_rf = RandomForestClassifier(class_weight='balanced')
model_rf.fit(tf_idf1, train['toxic'])
threshold_rf = 0.3
predictions_rf = np.where(model_rf.predict_proba(tf_idf2)[:,1] > threshold_rf, 1, 0)
f1_score(test['toxic'], predictions_rf)

0.6860220704151341

In [9]:
model_logreg = LogisticRegression(class_weight='balanced')
model_logreg.fit(tf_idf1, train['toxic'])
threshold_logreg = 0.7
predictions_logreg = np.where(model_logreg.predict_proba(tf_idf2)[:,1] > threshold_logreg, 1, 0)
f1_score(test['toxic'], predictions_logreg)

0.780725362040241

# 3. Выводы

В ходе работы над проектом мы провели очистку и лемматизацию текста, а также подсчитали величину TF-IDF для обучающей и тестовой выборок. На основе этих результатов нам удалось построить модель линейной регрессии, которая демонстрирует результат метрики f1 выше заданного порога (0.78 против 0.75).